In [13]:
from dotenv import load_dotenv
import os
import json
import textwrap

from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.docstore.document import Document
from langchain.chains import GraphCypherQAChain
from langchain_community.llms import Ollama
from langchain_neo4j.chains.graph_qa.cypher import GraphCypherQAChain

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [14]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_ENDPOINT = 'https://api.openai.com/v1/embeddings'

VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [15]:
graph = Neo4jGraph(
    url="bolt://localhost:7685",
    username="neo4j",
    password="myPassword"
)

In [16]:
llm = Ollama(
    model="llama3",  
    temperature=0,
)
llm_transformer = LLMGraphTransformer(llm=llm)

In [17]:
text = """
Nikola Tesla, born in 1856 in the small village of Smiljan in modern-day Croatia, was a visionary inventor, electrical engineer, and mechanical engineer whose ideas transcended borders and reshaped the world. Raised in a culturally diverse region of the Austro-Hungarian Empire, Tesla was influenced by the rich traditions of Croatian, Serbian, and Austrian communities, an experience that sparked his lifelong passion for innovation. He later pursued his studies in engineering and physics in Austria and France, where the vibrant academic and experimental environments further honed his groundbreaking ideas.

In his early career, Tesla's fascination with the unseen forces of nature led him to explore cutting-edge theories and conduct daring experiments that challenged conventional science. His relentless curiosity and innovative spirit drove him to delve into areas such as wireless transmission of energy, radio frequency experimentation, and the development of early robotics concepts, laying the groundwork for technologies that would only be fully appreciated decades later.

Tesla is renowned for his pioneering work in developing the alternating current (AC) electrical system, a transformative achievement that revolutionized global power distribution and set the stage for modern energy infrastructures. His innovations not only impacted the United States—where his inventions were widely adopted—but also left a lasting mark in other parts of Europe and beyond, including influential collaborations with scientists and engineers in Serbia, England, and Italy. His work fostered international dialogue in the scientific community, helping to bridge geographical and cultural divides, and inspiring future generations to challenge established norms.

Throughout his career, Tesla's journey intertwined several key nodes: his birthplace (Smiljan), his multifaceted professional roles (inventor, electrical engineer, mechanical engineer), and his seminal contributions (the AC power system, advancements in radio technology, and innovations in wireless communication). Moreover, his later endeavors into renewable energy concepts and his visionary proposals for harnessing natural forces have only deepened his legacy. These interconnected nodes illustrate how Tesla’s early experiences, diverse expertise, and international influences collectively forged a legacy that continues to inspire technological progress across the globe.
"""

docs = [Document(page_content=text)]


In [18]:
graph_documents = llm_transformer.convert_to_graph_documents(docs)
print(f"Derived Nodes:\n{graph_documents[0].nodes}\n")
print(f"Derived Edges:\n{graph_documents[0].relationships}")

Derived Nodes:
[Node(id='Nikola Tesla', type='Person', properties={}), Node(id='Alternating Current (AC) electrical system', type='Technology', properties={}), Node(id='Austria', type='Country', properties={}), Node(id='Scientists and engineers in Serbia, England, and Italy', type='People', properties={}), Node(id='Smiljan', type='Location', properties={}), Node(id='Croatian, Serbian, and Austrian communities', type='Cultures', properties={}), Node(id='France', type='Country', properties={}), Node(id='Austro-Hungarian Empire', type='Region', properties={})]

Derived Edges:
[Relationship(source=Node(id='Nikola Tesla', type='Person', properties={}), target=Node(id='Smiljan', type='Location', properties={}), type='BORN_IN', properties={}), Relationship(source=Node(id='Nikola Tesla', type='Person', properties={}), target=Node(id='Austro-Hungarian Empire', type='Region', properties={}), type='RAISED_IN', properties={}), Relationship(source=Node(id='Nikola Tesla', type='Person', properties={

In [19]:
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=[
        "Person",        # Individuals (e.g., Nikola Tesla)
        "Country",       # Countries (e.g., Croatia)
        "City",          # Cities (if applicable)
        "Village",       # Smaller settlements (e.g., Smiljan)
        "Organization",  # Institutions or companies
        "Profession",    # Roles (e.g., Inventor, Electrical Engineer)
        "Invention",     # Notable innovations (e.g., AC power system)
        "Date"           # Dates (e.g., birth date)
    ],
    allowed_relationships=[
        "BORN_IN",       # Birthplace relationship
        "NATIONALITY",   # Nationality association
        "LOCATED_IN",    # Geographic relationship (e.g., village located in country)
        "WORKED_AT",     # Employment relationship
        "WORKED_AS",     # Professional role relationship
        "SPOUSE",        # Spousal relationship
        "EDUCATED_AT",   # Education-related relationship
        "INVENTED"       # Relationship for innovations or inventions
    ]
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
    docs
)
print(f"Nodes:{graph_documents_filtered[0].nodes}")
print(f"Relationships:{graph_documents_filtered[0].relationships}")

Nodes:[Node(id='Smiljan', type='Village', properties={}), Node(id='Mechanical Engineer', type='Profession', properties={}), Node(id='Inventor', type='Profession', properties={}), Node(id='Alternating Current (AC) Electrical System', type='Invention', properties={}), Node(id='Scientists and Engineers in Serbia, England, and Italy', type='Organization', properties={}), Node(id='Electrical Engineer', type='Profession', properties={}), Node(id='Nikola Tesla', type='Person', properties={})]
Relationships:[Relationship(source=Node(id='Nikola Tesla', type='Person', properties={}), target=Node(id='Smiljan', type='Village', properties={}), type='BORN_IN', properties={}), Relationship(source=Node(id='Nikola Tesla', type='Person', properties={}), target=Node(id='Inventor', type='Profession', properties={}), type='WORKED_AS', properties={}), Relationship(source=Node(id='Nikola Tesla', type='Person', properties={}), target=Node(id='Electrical Engineer', type='Profession', properties={}), type='WORK

In [20]:
def print_graph_documents(graph_docs):
    for i, doc in enumerate(graph_docs):
        header_line = "=" * 60
        print(f"\n{header_line}\n Document {i+1}\n{header_line}")

        print("\nNodes:")
        for node in doc.nodes:
            print(f" ┌─ Node Type: {node.type}")
            if node.properties:
                for key, value in node.properties.items():
                    print(f" │   {key}: {value}")
            else:
                print(" │   (No properties)")
            print(" └" + "-" * 40)
                
        print("\nRelationships:")
        for rel in doc.relationships:
            print(f" ┌─ Relationship Type: {rel.type}")
            print(f" │   From: {rel.source}")
            print(f" │   To:   {rel.target}")
            if rel.properties:
                print(" │   Properties:")
                for key, value in rel.properties.items():
                    print(f" │      {key}: {value}")
            else:
                print(" │   (No properties)")
            print(" └" + "-" * 40)

# Call the function
# print_graph_documents(graph_documents_filtered)


# Alternatively, for a more compact view:
def print_compact_graph(graph_docs):
    for i, doc in enumerate(graph_docs):
        header_line = "=" * 60
        print(f"\n{header_line}\n Document {i+1}\n{header_line}")
        
        for rel in doc.relationships:
            arrow = "─" * 5
            print(f" ({rel.source}) {arrow}[{rel.type}]{arrow} ({rel.target})")
            if rel.properties:
                print(f"    Properties: {rel.properties}")

# Call the compact version
print_compact_graph(graph_documents_filtered)




 Document 1
 (id='Nikola Tesla' type='Person' properties={}) ─────[BORN_IN]───── (id='Smiljan' type='Village' properties={})
 (id='Nikola Tesla' type='Person' properties={}) ─────[WORKED_AS]───── (id='Inventor' type='Profession' properties={})
 (id='Nikola Tesla' type='Person' properties={}) ─────[WORKED_AS]───── (id='Electrical Engineer' type='Profession' properties={})
 (id='Nikola Tesla' type='Person' properties={}) ─────[WORKED_AS]───── (id='Mechanical Engineer' type='Profession' properties={})
 (id='Nikola Tesla' type='Person' properties={}) ─────[INVENTED]───── (id='Alternating Current (AC) Electrical System' type='Invention' properties={})


In [21]:
graph.add_graph_documents(graph_documents_filtered)

In [22]:
# Create the graph QA chain excluding Concept
graph_qa_chain = GraphCypherQAChain.from_llm(
    graph=graph, 
    llm=llm,
    allow_dangerous_requests = True,
    verbose=True 
)


In [23]:
# Invoke the chain with the input provided
result = graph_qa_chain.invoke({
    "query": "What Location was Nikola Tesla born ?"
})
print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...


Generated Cypher:
MATCH (n:Person {id: "Nikola Tesla"})-[:BORN_IN]->(m:Location) RETURN m.id AS location_id;
Full Context:
[{'location_id': 'Smiljan'}]

> Finished chain.
Final answer: Smiljan.


In [24]:
# Invoke the chain with the input provided
result = graph_qa_chain.invoke({
    "query": "Where did Nikola Tesla's work made impact?"
})
print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...


Generated Cypher:
MATCH (n:Person {id: "Nikola Tesla"})-[:IMPACTED]->(c:Country) RETURN c.id AS countryId;
Full Context:
[{'countryId': 'United States'}]

> Finished chain.
Final answer: The United States.


In [25]:
# Cell 3: Query the professional roles held by Nikola Tesla
result = graph_qa_chain.invoke({
    "query": "What professional roles did Nikola Tesla have?"
})
print(f"Final answer: {result['result']}")




> Entering new GraphCypherQAChain chain...


Generated Cypher:
MATCH (p:Person {id: "Nikola Tesla"})-[:WORKED_AS]->(pr:Profession) RETURN pr;
Full Context:
[{'pr': {'id': 'Mechanical Engineer'}}, {'pr': {'id': 'Electrical Engineer'}}, {'pr': {'id': 'Inventor'}}]

> Finished chain.
Final answer: Nikola Tesla had the professional roles of Mechanical Engineer, Electrical Engineer, and Inventor.


In [26]:
# Cell 4: Query the major inventions or innovations attributed to Nikola Tesla
result = graph_qa_chain.invoke({
    "query": "What are the major inventions or innovations attributed to Nikola Tesla?"
})
print(f"Final answer: {result['result']}")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: "Nikola Tesla"})-[:INVENTED]->(i:Invention) RETURN i;
Full Context:
[{'i': {'id': 'Alternating Current (AC) electrical system'}}, {'i': {'id': 'Alternating Current (AC) Electrical System'}}]

> Finished chain.
Final answer: Nikola Tesla is attributed to Alternating Current (AC) electrical system.


In [27]:
# Cell 5: Query the relationships and collaborations involving Nikola Tesla
result = graph_qa_chain.invoke({
    "query": "What collaborations or relationships did Nikola Tesla have with other entities?"
})
print(f"Final answer: {result['result']}")




> Entering new GraphCypherQAChain chain...


Generated Cypher:
MATCH (n:Person {id: "Nikola Tesla"})-[:COLLABORATED_WITH]-(g:Group) RETURN g;
Full Context:
[{'g': {'id': 'Scientists and engineers in Serbia, England, and Italy'}}]

> Finished chain.
Final answer: Serbia, England, and Italy.
